# Extended repressilator

This repressilator includes additional "steps" - signalling molecules that act as intermediates between repressors.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1.02, init_concentration=10**-7)
tetr = lc.Regulator(name='TetR', degradation_rate=1.02)
ci = lc.Regulator(name='cI', degradation_rate=1.02)

# Signalling molecules
c4 = lc.Regulator(name='C4', degradation_rate=0.0133)
scb2 = lc.Regulator(name='SCB2', degradation_rate=0.108)
c14 = lc.Regulator(name='C14', degradation_rate=0.017)

genetic_n.add_regulator([tetr, laci, ci, c4, scb2, c14])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1.02)
scbb = lc.Regulator(name='ScbB', degradation_rate=1.02)
cini = lc.Regulator(name='CinI', degradation_rate=1.02)
genetic_n.add_regulator([rhli, scbb, cini])

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=0.372, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=0.372, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=0.372, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
# #First extended node
# c4_laci = lc.Hill1(name='Prhl', input=c4, output=laci, alpha=[1.61*10**-8, 2.74*10**-7], K=5.49*10**-10, n=1.05)
# laci_not_scb2 = lc.Hill1(name='Plac', input=laci, output=[scb2, mcherry], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)

# # Second
# scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=tetr, alpha=[1.61*10**-8, 2.74*10**-7], K=5.49*10**-10, n=1.05)
# tetr_not_c14 = lc.Hill1(name='Ptet', input=tetr, output=[c14, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)

# # Third
# c14_ci = lc.Hill1(name='Pcin', input=c14, output=ci, alpha=[1.61*10**-8, 2.74*10**-7], K=5.49*10**-10, n=1.05)
# ci_not_c4 = lc.Hill1(name='PcI', input=ci, output=[c4, cyan], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)


# genetic_n.add_operator([c4_laci, laci_not_scb2, scb2_tetr, tetr_not_c14, c14_ci, ci_not_c4])

In [ ]:
# option with producers

#First extended node
c4_laci = lc.Hill1(name='Prhl', input=c4, output=laci, alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)
laci_not_scbb = lc.Hill1(name='Plac', input=laci, output=[scbb, mcherry], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
scbb_scb2 = lc.Producer(name='ScbB', enzyme=scbb, product=scb2, k2=66)

# Second
scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=tetr, alpha=[1.61*10**-8, 2.74*10**-7], K=5.49*10**-10, n=1.05)
tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[cini, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
cini_c14 = lc.Producer(name='CinI', enzyme=cini, product=c14, k2=60)

# Third
c14_ci = lc.Hill1(name='Pcin', input=c14, output=ci, alpha=[2.15*10**-8, 7.35*10**-8], K=3.41*10**-6, n=1.87)
ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=[rhli, cyan], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
rhli_c4 = lc.Producer(name='ScbB', enzyme=rhli, product=c4, k2=960)

genetic_n.add_operator([c4_laci, laci_not_scbb, scb2_tetr, tetr_not_cini, c14_ci, ci_not_rhli])
genetic_n.add_impactor([scbb_scb2, cini_c14, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
genetic_n.draw(pos=nx.planar_layout)

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=50, stochastic=False, mode="track_all")

In [ ]:
m = assay.measurements
# fig,(ax, ax1, ax2) = plt.subplots(3,1, sharex=True)
fig,(ax, ax1, ax2, ax3) = plt.subplots(4,1, sharex=True)
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='SCB2 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='ScbB in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='CinI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax1.legend(['LacI', 'TetR', 'cI'])
ax2.legend(['C4', 'SCB2', 'C14'])
ax3.legend(['RhlI', 'ScbB', 'CinI'])

Then I can also change how each signal diffuses (to see how that affects the system)

In [ ]:
assay.run(substeps=50, stochastic=False, mode='single_cell')

In [ ]:
m = assay.measurements
fig, ax = plt.subplots(1, 1)
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
ax.legend(['mCherry', 'CFP', 'mVenus'])

In [ ]:
assay.run(substeps=50, stochastic=False)

In [ ]:
m = assay.measurements
fig, ax = plt.subplots(1, 1)
m[m.Signal=='mCherry'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
ax.legend(['mCherry', 'CFP', 'mVenus'])


# With AiiA

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1.02)
genetic_n.add_regulator(aiia)

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=[c4, c14, scb2], Km=[0.0008, 0.00015, 0.303], k2=[1.76*10**6, 2.34*10**6, 2.12*10**4])
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, aiia], alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)
scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=[tetr, aiia], alpha=[1.61*10**-8, 2.74*10**-7], K=5.49*10**-10, n=1.05)
c14_ci = lc.Hill1(name='Pcin', input=c14, output=[ci, aiia], alpha=[2.15*10**-8, 7.35*10**-8], K=3.41*10**-6, n=1.87)
genetic_n.add_impactor(lactonase)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=50, stochastic=False, mode="track_all")

In [ ]:
m = assay.measurements
# fig,(ax, ax1, ax2) = plt.subplots(3,1, sharex=True)
fig,(ax, ax1, ax2, ax3) = plt.subplots(4,1, sharex=True)
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='SCB2 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='ScbB in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='CinI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax1.legend(['LacI', 'TetR', 'cI'])
ax2.legend(['C4', 'SCB2', 'C14'])
ax3.legend(['RhlI', 'ScbB', 'CinI', 'AiiA'])

In [ ]:
assay.run(substeps=50, stochastic=False, mode='single_cell')

In [ ]:
m = assay.measurements
fig, ax = plt.subplots(1, 1)
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
ax.legend(['mCherry', 'CFP', 'mVenus'])

In [ ]:
# assay.run(substeps=50, stochastic=False)

In [ ]:
# m = assay.measurements
# fig, ax = plt.subplots(1, 1)
# m[m.Signal=='mCherry'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
# m[m.Signal=='CFP'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
# m[m.Signal=='mVenus'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
# ax.legend(['mCherry', 'CFP', 'mVenus'])
